In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cudf as pd
import cupy as np
import cuml as cuml

In [3]:
TRAIN_PATH = "../input/house-prices-advanced-regression-techniques/train.csv"
TEST_PATH = "../input/house-prices-advanced-regression-techniques/test.csv"
SUBMISSION_PATH = "submission.csv"
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
sub=pd.DataFrame(test_df['Id'])
train_df.head()

In [4]:
print(train_df.shape)
print(test_df.shape)
train_df.describe()

In [5]:
train_df['SalePrice'].dtype

In [6]:
train_df.columns

In [7]:
train_df.isnull().any().sum()

In [8]:
nulls=train_df.isnull().sum()
nulls[nulls!=0].sort_values(ascending=False)

In [9]:
train_df=train_df.drop(['Id','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1)
test_df=test_df.drop(['Id','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1)

In [10]:
y = train_df['SalePrice']
train_df.drop('SalePrice',axis=1,inplace=True)

In [11]:
for col in train_df:
    if train_df[col].dtype=='O':
        train_df[col].fillna(train_df[col].mode()[0], inplace=True)
        test_df[col].fillna(test_df[col].mode()[0], inplace=True)
    else:
        train_df[col].fillna(train_df[col].mean(), inplace=True)
        test_df[col].fillna(test_df[col].mean(), inplace=True)


In [12]:
train_df.isnull().any().sum()

In [13]:
from cuml.preprocessing import LabelEncoder
obj_cols=list(train_df.select_dtypes(['object']).columns)
le=LabelEncoder()
for col in obj_cols:
    train_df[col]=le.fit_transform(train_df[col])
    test_df[col]=le.fit_transform(test_df[col])


In [14]:
train_df.shape

In [15]:
test_df.shape

In [16]:
from cuml.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled = scaler.fit_transform(train_df)
test_df=scaler.transform(test_df)

In [17]:
from cuml.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_scaled, y, test_size = 0.3, random_state = 42)

In [18]:
from cuml.linear_model import LinearRegression
algorithm = ['svd','svd-jacobi', 'eig', 'svd-qr','qr']
MSE=[]
R2_Score=[]
MAE =[]
for alg in algorithm:
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = alg)
    reg = lr.fit(X_train,Y_train)
    predict = lr.predict(X_test)
    MSE.append(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),predict.astype('int64')))
    R2_Score.append(cuml.metrics.regression.r2_score(Y_test.astype('float32'),predict.astype('float32')))
    MAE.append(cuml.metrics.regression.mean_absolute_error(Y_test.astype('int64'),predict.astype('int64')))

In [19]:
#lasso
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.1)
result_lasso = ls.fit(X_train,Y_train)
predict = result_lasso.predict(X_test)
print(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),predict.astype('int64')))
print(cuml.metrics.regression.r2_score(Y_test.astype('float32'),predict.astype('float32')))
print(cuml.metrics.regression.mean_absolute_error(Y_test.astype('int64'),predict.astype('int64')))

In [20]:
#ridge
from cuml import Ridge
from cuml.linear_model import Ridge

alpha = np.array([1e-5])
ridge_eig = Ridge(alpha = alpha, fit_intercept = True, normalize = False,
              solver = "eig")
result_ridge_eig = ridge_eig.fit(X_train,Y_train)
predict = result_ridge_eig.predict(X_test)
print("rdge eig")
print(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),predict.astype('int64')))
print(cuml.metrics.regression.r2_score(Y_test.astype('float32'),predict.astype('float32')))
print(cuml.metrics.regression.mean_absolute_error(Y_test.astype('int64'),predict.astype('int64')))
result_ridge_eig = ridge_eig.fit(X_train,Y_train)
ridge_svd = Ridge(alpha = alpha, fit_intercept = True, normalize = False,
              solver = "svd")
result_ridge_svd = ridge_svd.fit(X_train,Y_train)
print("ridge svd")
predict = result_lasso.predict(X_test)
print(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),predict.astype('int64')))
print(cuml.metrics.regression.r2_score(Y_test.astype('float32'),predict.astype('float32')))
print(cuml.metrics.regression.mean_absolute_error(Y_test.astype('int64'),predict.astype('int64')))


In [21]:
final = pd.DataFrame(index = algorithm)
final['MSE']=MSE
final['R2_Score']=R2_Score
final['MAE']=MAE
final.to_csv('results.csv')
final

In [22]:
ID = "Id"
TARGET = "SalePrice"
sub[TARGET] = lr.predict(test_df)
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()